In [1]:
!pip install torch==1.2.0 torchvision==0.4.0

     |████████████████████████████████| 748.9 MB 623 bytes/s 
     |████████████████████████████████| 8.8 MB 66.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.2.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/MyDrive/6D-pose-project")

In [ ]:
import torch
print(torch.__version__)

1.2.0


In [4]:
from lib.network import PoseNet, PoseRefineNet
from lib.loss import Loss
from lib.utils import setup_logger
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
import torch
import os
from torch import optim
import numpy as np
import time
from torch.autograd import Variable
%load_ext autoreload
%autoreload 2


In [5]:
import warnings
warnings.filterwarnings("ignore") 

In [6]:
num_objects = 13
num_points = 500
outf = 'trained_models/linemod'
log_dir = 'experiments/logs/linemod'
repeat_epoch = 20
decay_margin = 0.016
decay_start = False
lr_rate = 0.3
w_rate = 0.3

In [7]:
model = 'trained_models/linemod/pose_model_current.pth'
estimator = PoseNet(num_points = num_points, num_obj = num_objects)
estimator.load_state_dict(torch.load(model))
estimator.cuda();

In [8]:
estimator

PoseNet(
  (cnn): ModifiedResnet(
    (model): DataParallel(
      (module): PSPNet(
        (feats): ResNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (relu): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (relu): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
            (1): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (relu): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
          )
          (layer2): Sequential(
            (

In [ ]:
lr = 0.0001

optimizer = optim.Adam(estimator.parameters(), lr=lr)

In [ ]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start)

dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=4)

Object 1 buffer loaded
Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 6 buffer loaded
Object 8 buffer loaded
Object 9 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded
Object 12 buffer loaded
Object 13 buffer loaded
Object 14 buffer loaded
Object 15 buffer loaded


In [ ]:
test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start)
testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=4)

Object 1 buffer loaded
Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 6 buffer loaded
Object 8 buffer loaded
Object 9 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded
Object 12 buffer loaded
Object 13 buffer loaded
Object 14 buffer loaded
Object 15 buffer loaded


In [ ]:
sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

In [ ]:
print('>>>>>>>>----------Dataset loaded!---------<<<<<<<<\nlength of the training set: {0}\nlength of the testing set: {1}\nnumber of sample points on mesh: {2}\nsymmetry object list: {3}'
      .format(len(dataset), 
              len(test_dataset), 
              num_points_mesh, 
              sym_list))

>>>>>>>>----------Dataset loaded!---------<<<<<<<<
length of the training set: 2373
length of the testing set: 1336
number of sample points on mesh: 500
symmetry object list: [3, 4]


In [ ]:
criterion = Loss(num_points_mesh, sym_list)

In [ ]:
start_epoch = 25
nepoch = 30
w = 0.015
batch_size = 8

In [ ]:
next(iter(dataloader))[1].shape

torch.Size([1, 1, 500])

In [ ]:
best_test = np.Inf
print_every = 50

if start_epoch == 1:
    for log in os.listdir(log_dir):
        if '.ipyn' not in log:
            os.remove(os.path.join(log_dir, log))
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    logger = setup_logger('epoch%d' % epoch, os.path.join(log_dir, 'epoch_%d_log.txt' % epoch))
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    logger.info(mess)
    
    train_count = 0
    train_dis_avg = 0.0
    
    estimator.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data        
            #points[0, :, 2] = 0 
            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
            loss, dis, new_points, new_target = criterion(pred_r, 
                                                          pred_t, 
                                                          pred_c, 
                                                          target, 
                                                          model_points, 
                                                          idx, 
                                                          points, 
                                                          w, refine_start)
            loss.backward()
            
            train_dis_avg += dis.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_dis:{4}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size)
                logger.info(mess)
                
                optimizer.step()
                optimizer.zero_grad()
                train_dis_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
                torch.save(estimator.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_count = 0
    estimator.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        #points[0, :, 2] = 0 
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
        pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
        _, dis, new_points, new_target = criterion(pred_r, pred_t, pred_c, 
                                                   target, 
                                                   model_points, idx, points, 
                                                   w, refine_start)
        test_dis += dis.item()
        mess = 'Test time {0} Test Frame No.{1} dis:{2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), test_count, dis)
        logger.info(mess)
        
        test_count += 1
    test_dis = test_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), epoch, test_dis)
    print(mess)
    logger.info(mess)
    
    if test_dis <= best_test:
        best_test = test_dis
        torch.save(estimator.state_dict(), '{0}/pose_model_{1}_{2}.pth'.format(outf, epoch, test_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        
    if best_test < decay_margin and not decay_start:
        decay_start = True
        lr *= lr_rate
        w *= w_rate
        optimizer = optim.Adam(estimator.parameters(), lr=lr)

Train time 00h 00m 00s, Training started
Train time 00h 02m 08s Epoch 25 Batch 25 Frame 200 Avg_dis:0.007828020781744272
Train time 00h 03m 16s Epoch 25 Batch 50 Frame 400 Avg_dis:0.005336579110007733
Train time 00h 04m 23s Epoch 25 Batch 75 Frame 600 Avg_dis:0.006870315410196781
Train time 00h 05m 29s Epoch 25 Batch 100 Frame 800 Avg_dis:0.006819469825131819
Train time 00h 06m 34s Epoch 25 Batch 125 Frame 1000 Avg_dis:0.006178461364470422
Train time 00h 07m 41s Epoch 25 Batch 150 Frame 1200 Avg_dis:0.005977761669782922
Train time 00h 08m 46s Epoch 25 Batch 175 Frame 1400 Avg_dis:0.005816808727104217
Train time 00h 09m 52s Epoch 25 Batch 200 Frame 1600 Avg_dis:0.005799111328087747
Train time 00h 10m 58s Epoch 25 Batch 225 Frame 1800 Avg_dis:0.004990249319234863
Train time 00h 12m 04s Epoch 25 Batch 250 Frame 2000 Avg_dis:0.006697529403027147
Train time 00h 13m 09s Epoch 25 Batch 275 Frame 2200 Avg_dis:0.007155830564443022
Train time 00h 14m 08s Epoch 25 Batch 300 Frame 2400 Avg_dis:0.0

In [ ]:
t = next(iter(dataloader))

In [ ]:
points, choose, img, target, model_points, idx = t

In [ ]:
points.shape

torch.Size([1, 500, 3])

In [ ]:
target.shape

torch.Size([1, 500, 3])

In [ ]:
choose.shape

torch.Size([1, 1, 500])

In [ ]:
model_points.shape

torch.Size([1, 500, 3])